## Test the MIRI imager distortion solution (standalone and JWST pipeline versions)  ##

### Set basic environment stuff ###

In [1]:
import os as os
import numpy as np
from numpy.testing import assert_allclose
import pdb as pdb

Import the MIRI coordinates code from https://github.com/STScI-MIRI/miricoord and ensure that it is on the PYTHONPATH.  Also ensure that the local data directory environmental variable is set:

setenv MIRICOORD_DATA_DIR /YourLocalPathToData/ (this is where io will happen)

In [2]:
data_dir=os.path.expandvars('$MIRICOORD_DATA_DIR')

### First we'll test the standalone distortion solution (mirim_tools) ###

In [3]:
import miricoord.miricoord.imager.mirim_tools as mt

Define a point on the imager (0-indexed convention) and convert to v2,v3 assuming F770W filter

In [4]:
x,y=692.5,511.5

In [5]:
v2,v3=mt.xytov2v3(x,y,'F770W')

In [6]:
print(v2,v3)

[-453.36336335] [-374.06863059]


The V2,V3 location of the imager reference point should be -453.363, -374.069

Convert back to x,y pixel coordinates

In [7]:
xback,yback=mt.v2v3toxy(v2,v3,'F770W')

In [8]:
print(xback,yback)

[692.49606433] [511.49995018]


The returned pixel coordinates should be the same as the original pixel coordinates to within a small fraction of a pixel (roundtrip error)

Now run a full set of tests using both F770W and F1800W filters.  If no errors, there were no problems:

In [9]:
mt.testtransform()

### Now we'll test the implementation of the distortion solution in the JWST pipeline (mirim_pipetools) ###

In [10]:
from astropy.modeling import models
from asdf import AsdfFile
from jwst import datamodels
from jwst.assign_wcs import miri

Point to a simulated F770W imager observation created with mirisim and reduced with the B7.1 pipeline through the end of assign_wcs

In [11]:
file=data_dir+'jwstb71testing/20171214_112906_mirisim/det_images/det_image_1_MIRIMAGE_F770Wexp1_assign_wcs.fits'

Read the datamodel from the file and print some information about it to see the available reference frames.

In [12]:
im = datamodels.ImageModel(file)

In [13]:
print(im.meta.wcs)

  From      Transform   
-------- ---------------
detector CompoundModel57
    v2v3        v23tosky
   world            None


In [14]:
print(im.meta.wcs.available_frames)

['detector', 'v2v3', 'world']


Define a point at the reference point of the imager and check that the v2,v3 is correct

In [15]:
x,y=692.5,511.5

In [16]:
v2,v3=im.meta.wcs.transform("detector", "v2v3", x,y)

In [17]:
print(v2*3600.,v3*3600.)

-453.3633633490669 -374.0686305853809


The V2,V3 location of the imager reference point should be -453.363, -374.069

The RA/DEC of the reference point is meant to be zero,zero in this simulation, check that it is

In [18]:
ra, dec = im.meta.wcs(x, y)

In [19]:
print(ra*3600,dec*3600)

5.975467107799582e-10 -3.3424456172422127e-14


Run a full suite of tests of the WCS using many locations in the field.  Define a set of reference points in V2V3.

In [20]:
import miricoord.miricoord.imager.mirim_pipetools as mpt

In [21]:
mpt.testtransform()

Test the bounding boxes (all of the following should be nan)

In [22]:
im.meta.wcs(3.49,500)

(nan, nan)

In [23]:
im.meta.wcs(1027.6,500)

(nan, nan)

In [24]:
im.meta.wcs(500,-0.6)

(nan, nan)

In [25]:
im.meta.wcs(500,1023.6)

(nan, nan)